# Setup Dependencies

In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet psycopg2 langchain langchain-community langchain-openai faiss-cpu langgraph python-dotenv
%pip install --extra-index-url https://europe-west3-python.pkg.dev/bionic-unity-294411/radicalbit-python-api/simple radicalbit-client==3.4.4

In [ ]:
def beautify_gpt_response(response):
    return response[list(response.keys())[0]]['messages'][0].pretty_repr()

# Setup Database

In [ ]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os

load_dotenv()

db_host = os.environ.get("DATABASE_HOST")
db_port = os.environ.get("DATABASE_PORT")
db_name = os.environ.get("DATABASE_NAME")
db_user = os.environ.get("DATABASE_USER")
db_password = os.environ.get("DATABASE_PASSWORD")

db = SQLDatabase.from_uri(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",
)
print(f"SQL dialect: {db.dialect}")
print(f"List of usable tables: {db.get_usable_table_names()}")

# Create Agent

In [ ]:
import os
api_key = os.environ.get("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

In [ ]:
import json
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for t in tools:
    print(json.dumps(t.to_json(), indent=4))

In [ ]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies that want to retrieve a specific number of examples they wish to obtain, always limit your query to at most 5 results.
If the user ask for all the examples do not limit the query to 5 result.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
ALWAYS use double quote to identify table name.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

# Test human language query

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="list all truck models of Renault")]}
):
    print(beautify_gpt_response(s))

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="list truck models of Renoul")]}
):
    print(beautify_gpt_response(s))

# Embrace the typos with Similarity Search

In [ ]:
from langchain.evaluation import load_evaluator
from langchain_openai import OpenAIEmbeddings

evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings=OpenAIEmbeddings())

print(f'Distance Amsterdam - coffeeshop: {evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")['score']}')
print(f'Distance Tokyo - coffeeshop: {evaluator.evaluate_strings(prediction="Tokyo", reference="coffeeshop")['score']}')

In [ ]:
import ast
import re

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

names = query_as_list(db, "SELECT name FROM fleet_model")
manufacturers = query_as_list(db, "SELECT manufacturer FROM fleet_model")
colors = query_as_list(db, "SELECT color FROM vehicle")
zone = query_as_list(db, "SELECT zone FROM vehicle_position")
city = query_as_list(db, "SELECT city FROM vehicle_position")
manufacturers[:5]

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(names + manufacturers + colors + zone + city, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

for doc, score in vector_db.similarity_search_with_score("Renoul"):
    print(f"* Distance from {doc.page_content} to Renoul = {score:3f}")

In [ ]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
If the user ask for all the examples do not limit the query to 5 result.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, state_modifier=system_message)

for t in tools:
    print(json.dumps(t.to_json(), indent=4))

In [ ]:
for s in agent.stream(
        {"messages": [HumanMessage(content="list truck models of Renoul")]}
):
    print(beautify_gpt_response(s))

# Enhance response with Radicalbit RAG

In [ ]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many models manufactured by Scania exist?")]}
):
    print(beautify_gpt_response(s))

In [ ]:
from radicalbit_client.rag import RadicalbitRagClient
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.environ.get("RB_CLIENT_ID")
client_secret = os.environ.get("RB_CLIENT_SECRET")

# Create and instance of the Radicalbit RAG Client
radicalbit_rag_client = RadicalbitRagClient(
    host="platform.radicalbit.ai",
    port ="443",
    client_id=client_id,
    client_secret=client_secret,
    tenant_name="bitrockcodemotion2024"
)

agent_answer = s["agent"]["messages"][-1].content

payload = radicalbit_rag_client.invoke(prompt=agent_answer)
print(payload['answer'])